In [1]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp
import cmath
import time

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 
from scipy.linalg import expm

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize
from random import randint, uniform, random

In [2]:
# Manipulação das matrizes, soma e subtração

def somar(A, B):
    C = []
    num_linhas_a = len(A)
    num_colunas_a = len(A[0])
    
    for i in range (num_linhas_a):
        linha = [0]*num_colunas_a
        C.append(linha)
        for j in range(num_colunas_a):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    num_linhas_a = len(A)
    num_colunas_a = len(A[0])
    
    for i in range (num_linhas_a):
        linha = [0]*num_colunas_a
        C.append(linha)
        for j in range(num_linhas_a):
            C[i][j] = A[i][j] - B[i][j]

    return C

In [3]:
## CONSTANTES PARA O CONTROLADOR

# Considerações: constante de Planck verdadeira: 1, massa: 1

TEMPO_ANALISE = 10 
QUANTIDADE_PONTOS_AMOSTRAGEM = 100
PASSO = TEMPO_ANALISE/QUANTIDADE_PONTOS_AMOSTRAGEM

MIN_HORIZONTE = 3 # Horizonte mínimo da análise
MAX_HORIZONTE = 30 # Horizonte máximo da análise

# Com horizonte em 10, a avaliação dura em torno de 80 minutos 

DESEJADO = [[1.00],[0.00]] # [[c1];[c2]] = [[0.707],[-0.707]]
RHO_DESEJADO = np.matmul(DESEJADO, np.transpose(DESEJADO))
print(RHO_DESEJADO)

# Determinação dos valores de início e fim da análise
num = PASSO
contador = 0
while num < 1:
    num *= 10
    contador += 1

EXPOENTE = contador
INICIO_ANALISE = 0*10**(-EXPOENTE) # Tempo inicial da analise em um horizonte
FINAL_ANALISE = (1*10**(-EXPOENTE))+PASSO # Tempo final da analise em um horizonte

[[1. 0.]
 [0. 0.]]


In [4]:
## CONSTANTES PARA O ALGORITMO GENÉTICO

ELITISMO = True
PORCENTAGEM_ELITISMO = 0.20

TAMANHO_POP = 50
TAXA_MUTACAO = 0.10
TAXA_CRUZAMENTO = 0.70
GERACOES = 100

# Valores mínimos e máximos para gerar uma população
MIN = -5
MAX = 5

In [5]:
def dpsi_dt(t, ro, H): # A derivada da ro em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    M = [[0, -ro[0][1]], [-ro[1][0], 0]]
    A = (np.dot(H,ro))-(np.dot(ro, H))# [H,p]
    return -1j*A

# Runge-Kutta de quarta ordem

def runge_kutta(onda, fator_runge_kutta, hamiltoniano, tempo_inicial = 0):

    k1 = dpsi_dt(tempo_inicial, onda, hamiltoniano)
    k2 = dpsi_dt(tempo_inicial + 0.5 * fator_runge_kutta, somar(onda, np.dot((0.5*fator_runge_kutta), k1)), hamiltoniano)
    k3 = dpsi_dt(tempo_inicial + 0.5 * fator_runge_kutta, somar(onda, np.dot((0.5*fator_runge_kutta), k2)), hamiltoniano)
    k4 = dpsi_dt(tempo_inicial + fator_runge_kutta, somar(onda, np.dot(fator_runge_kutta, k3)), hamiltoniano)
    
    ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
    
    A = somar(np.dot(2,k3), k4)
    B = somar(np.dot(2,k2), k1)
    C = somar(A, B)

    runge = somar(onda,np.dot((fator_runge_kutta / 6.0),(C)))
    
    return runge

In [6]:
def func_objetivo(vetor_controles, tempo_atual, rho_desejado, 
                  rho_atual, H, horizonte, fator_runge_kutta):
    
    fo = 0
    pos = 0
    
    tempo_inicial = INICIO_ANALISE
    tempo_final = FINAL_ANALISE
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, complex(vetor_controles[0], vetor_controles[1])], 
                 [complex(vetor_controles[0], -vetor_controles[1]), 0]]
    
    matriz_inicial = rho_atual

    matriz_desejada = np.zeros((2,2),dtype=np.complex_)
    hamiltoniano = H

    H = [[np.exp(-1j*np.pi*tempo_atual/2),0],[0,np.exp(1j*np.pi*tempo_atual/2)]]
    U = H
    U = np.asmatrix(U)
    U_dagger = U.getH()
    teste = np.dot(np.dot(U,rho_desejado),U_dagger)
    matriz_desejada = np.squeeze(np.asarray(teste))

    matriz_teste = sub(matriz_inicial, matriz_desejada)
    
    teste = []
    for m in range(0, 2):
        teste.append(abs(matriz_teste[0][m])+abs(matriz_teste[0][m]))
    
    fo += max(teste)

    runge = runge_kutta(matriz_inicial, fator_runge_kutta, 
                        somar(hamiltoniano, controles), tempo_inicial = tempo_inicial)

    matriz_inicial = runge

    tempo_inicial = round(tempo_inicial+PASSO, EXPOENTE)
    tempo_final = round(tempo_final+PASSO, EXPOENTE)
    tempo = round(tempo_atual+PASSO, EXPOENTE)
    pos += 2

    while horizonte > 1:
        
        H = [[np.exp(-1j*np.pi*tempo/2),0],[0,np.exp(1j*np.pi*tempo/2)]]
        U = H
        U = np.asmatrix(U)
        U_dagger = U.getH()
        teste = np.dot(np.dot(U,rho_desejado),U_dagger)
        matriz_desejada = np.squeeze(np.asarray(teste))
        
        teste = []
        for m in range(0, 2):
            teste.append(abs(matriz_teste[0][m])+abs(matriz_teste[0][m]))

        matriz_teste = sub(matriz_inicial, matriz_desejada)
        
        fo += max(teste)
        
        controles = [[0, complex(vetor_controles[pos], vetor_controles[pos+1])], 
                     [complex(vetor_controles[pos], -vetor_controles[pos+1]), 0]]

        runge = runge_kutta(matriz_inicial, fator_runge_kutta, 
                        somar(hamiltoniano, controles), tempo_inicial = tempo_inicial)

        matriz_inicial = runge

        tempo_inicial = round(tempo_inicial+PASSO, EXPOENTE)
        tempo_final = round(tempo_final+PASSO, EXPOENTE)
        tempo = round(tempo_atual+PASSO, EXPOENTE)
        horizonte -= 1

    return fo

In [7]:
def main(horizonte, rho_inicial):
    
    u = [] # Controles 
    for i in range(0, 2*horizonte):
        u.append(uniform(-1,1))

    evolucao_1 = np.zeros((2,QUANTIDADE_PONTOS_AMOSTRAGEM), dtype=np.complex_)
    evolucao_2 = np.zeros((2,QUANTIDADE_PONTOS_AMOSTRAGEM), dtype=np.complex_)
        
    # O que será retornado por essa função
    
    vetor_controles = np.zeros((2, QUANTIDADE_PONTOS_AMOSTRAGEM), dtype=np.complex_) # Valores de controle
    
    H = np.zeros((2,2), dtype=np.complex_)
    H = [[(3/2*np.pi), 0], [0, (1/2*np.pi)]]
    
    tempo = 0
    pos = 0
    while tempo < TEMPO_ANALISE:

        evolucao_1[0][pos] = rho_inicial[0][0]
        evolucao_1[1][pos] = rho_inicial[1][1]

        evolucao_2[0][pos] = rho_inicial[0][1]
        evolucao_2[1][pos] = rho_inicial[1][0]

        bounds = [(-1,1)]
    
        resultado = optimize.minimize(func_objetivo, u, args=(tempo, RHO_DESEJADO, rho_inicial, H, horizonte, PASSO), bounds=bounds, method='L-BFGS-B')
        
        controle = np.zeros((2,2), dtype=np.complex_)

        vetor_controles[0][pos] = resultado.x[0]
        vetor_controles[1][pos] = resultado.x[1]

        print(f"Controles: {complex(resultado.x[0],resultado.x[1])} e {complex(resultado.x[0],-resultado.x[1])}\n-------------------------")
        
        controle = [[0, complex(resultado.x[0], resultado.x[1])],[complex(resultado.x[0], -resultado.x[1]), 0]]
        H = somar(H, controle)
    
        ## Adaptação da onda ao controle
        
        rho_avancado = runge_kutta(rho_inicial, PASSO, H, tempo_inicial = INICIO_ANALISE)
        rho_inicial = rho_avancado
        
        # Reset
        H = [[(3/2*np.pi), 0], [0, (1/2*np.pi)]]
    
        u = [] # Controles 
        for i in range(0, 2*horizonte):
            u.append(resultado.x[0])
            u.append(resultado.x[1])
        
        pos += 1
        tempo = round(tempo+PASSO, EXPOENTE)

    #teste = [[0.5, 0.5], [0.5, 0.5]]
    #rho_inicial = [[0.58, 0.5454], [0.5454, 0.42]]
    #print(f'O valor é: {np.sqrt(np.sqrt(rho_inicial)*teste*np.sqrt(rho_inicial))}')
    p#rint(f'Mais testes: {np.trace(np.sqrt(np.sqrt(rho_inicial)*teste*np.sqrt(rho_inicial)))}')
    print(f'O rho alcançado foi: {rho_inicial}')
    print(f'A fidelidade do experimento é de: {np.trace(np.sqrt(np.sqrt(rho_inicial)*teste*np.sqrt(rho_inicial)))**2}')
    return evolucao_1, evolucao_2, vetor_controles
    

In [8]:
# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

def plot_componentes(valores_onda,  evolucao_d, componente):
    
    ts1 = np.linspace(0, TEMPO_ANALISE, QUANTIDADE_PONTOS_AMOSTRAGEM)
    ts2 = np.linspace(0, TEMPO_ANALISE, QUANTIDADE_PONTOS_AMOSTRAGEM)
    
    real_primeiro = [elemento.real for elemento in valores_onda[componente-1]]
    imaginario_primeiro = [elemento.imag for elemento in valores_onda[componente-1]]
    
    real_segundo = [elemento.real for elemento in evolucao_d[componente-1]]
    imaginario_segundo = [elemento.imag for elemento in evolucao_d[componente-1]]
    
    # Plotagem
    plt.subplot(2,1,1)

    plt.ylabel('Pesos') 
    plt.xlabel('Tempo') 
    plt.title(f"{componente}ª coluna da matriz de densidade -> real")
    plt.plot(ts1, real_primeiro, color='r', marker='*', linewidth=1, markersize=1, label="primeiro")
    plt.plot(ts2, real_segundo, color='b', marker='o',  linewidth=1, markersize=1, label="segundo")
    plt.legend(loc='upper right')

    plt.axis([0,TEMPO_ANALISE,-1,1])
    
    plt.subplot(2,1,2)

    plt.ylabel('Imaginário') 
    plt.xlabel('Tempo') 
    plt.title(f"{componente}ª coluna da matriz de densidade -> imaginário")
    plt.plot(ts1, imaginario_primeiro, color='r', marker='*', linewidth=1, markersize=1, label="primeiro")
    plt.plot(ts2, imaginario_segundo, color='b', marker='o',  linewidth=1, markersize=1, label="segundo")
    plt.legend(loc='upper right')

    plt.axis([0,TEMPO_ANALISE,-1,1])

    plt.tight_layout()
    

In [9]:
PSI = np.ones((2,1), dtype=np.complex_)
PSI = [[0.00],[1.00]]
ro = np.matmul(PSI, np.transpose(PSI))
evolucao_1, evolucao_2, vetor_controles = main(4, ro)

Controles: (-1-1j) e (-1+1j)
-------------------------
Controles: (-1-1j) e (-1+1j)
-------------------------
Controles: (-1-1j) e (-1+1j)
-------------------------
Controles: (-1-1j) e (-1+1j)
-------------------------
Controles: (-1-1j) e (-1+1j)
-------------------------
Controles: (-1+1j) e (-1-1j)
-------------------------
Controles: (-1+1j) e (-1-1j)
-------------------------
Controles: (-1+1j) e (-1-1j)
-------------------------
Controles: (-1+1j) e (-1-1j)
-------------------------
Controles: (0.24427014452320225+1j) e (0.24427014452320225-1j)
-------------------------
Controles: (0.9242047595157921+1j) e (0.9242047595157921-1j)
-------------------------
Controles: (1+1j) e (1-1j)
-------------------------
Controles: (0.2573616835759926+0.668150724019648j) e (0.2573616835759926-0.668150724019648j)
-------------------------
Controles: (-0.0003813669309302876-0.000338132974420599j) e (-0.0003813669309302876+0.000338132974420599j)
-------------------------
Controles: (0.0002040270

NameError: name 'p' is not defined

In [ ]:
plot_componentes(evolucao_1, evolucao_2, 1)

In [ ]:
plot_componentes(evolucao_1, evolucao_2, 2)

In [ ]:
def acao_controle(valor, type):
    ts = np.linspace(0, TEMPO_ANALISE, QUANTIDADE_PONTOS_AMOSTRAGEM)

    plt.ylabel('Ação de controle')
    plt.xlabel('Tempo')
    plt.plot(ts, valor, color='k', marker='o', linewidth=1, markersize=1, label="Valor da ação de controle")
    plt.legend(loc="upper right")
    plt.title(f"Ação de controle {type}")
    plt.show()

In [ ]:
acao_controle(vetor_controles[0].real, "Real")

In [ ]:
acao_controle(vetor_controles[1].real, "Imaginário")